# Pracownia z Analizy Numerycznej

## Zadanie 2.16

## Autor : Mateusz Markiewicz

# Algorytm Clenshawa

Funkcja obliczająca wartość sumy Sn(x) =  0.5\*c0\*T0(x) + c1\*T1(x) + c2\*T2(x) + ... + cn\*Tn(x), dla zadanych wartości n,c0...cn,x

In [1]:
function clenshaw(n,c,x)
    Bk_1 = 0
    Bk_2 = 0
    for i = n:(-1):1
        Bk = Float64(2) * x * Bk_1 - Bk_2 + c[i+1]
        Bk_2 = Bk_1
        Bk_1 = Bk
    end
    return x * Bk_1 - Bk_2 + c[1]/Float64(2)
end

clenshaw (generic function with 1 method)

In [2]:
using Distributions

Funkcja zaburzająca wartość danego parametru o losową wartość epsilon z przedziału [-2^(-48),2^(-48)]

In [3]:
function fluct(c)
    return Float64(c) + rand(Uniform(-(2^(-48)),2^(-48)))
end

fluct (generic function with 1 method)

Funkcaj obliczająca wartość k-tego wielomiany czebyszewa Tk, w punkcie x z przedziału [-1,1]

In [4]:
function czebyszew(k,x)
    return cos(k * acos(x))
end

czebyszew (generic function with 1 method)

In [5]:
function czebyszew2(k,x)
    if k == 0
        return 1
    elseif k == 1
        return x
    else
        return 2*x*czebyszew(k-1,x) - czebyszew(k-2,x)
    end
end

czebyszew2 (generic function with 1 method)

Funkcja obliczająca ilość cyfr znaczących

In [6]:
function sign_figures(a,b)
    return -log(10,abs((a-b)/b))
end

sign_figures (generic function with 1 method)

Test dokładności obliczania Tn(x) dla zadanego n (20) oraz x z przedziału [-1,1] (parametry a0 oraz b oznaczaja odpowiednio początek i koniec przedziału i mogę być zmieniane, tak samo maxiter - liczba iteracji)

In [7]:
a0 = -1
a = a0
b = 1
max_iter = 50
e = (b-a)/max_iter
res = []
for i = 0:(max_iter-1)
    y0 = czebyszew(20,a)
    y = clenshaw(20,[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],a)
    s = sign_figures(y,y0)
    push!(res,s)
    a += e
end
    

In [8]:
using Plots
plotly()
x1 = a0:e:(b-e)
plot(x1,res,ylims = (0,20),seriestype=:scatter,title="Dokładność obliczania T20(x)")
xlabel!("x")
ylabel!("Ilość cyfr znaczących wyniku")

Funkcaj obliczająca błąd względny wyniku

In [9]:
function rel_err(v,av)
    return Float64(abs((v-av)/v))
end

rel_err (generic function with 1 method)

Test numerycznej poprawności algorytmu Clenshawa dla ck = k^(0.5)

Parametry n, a0, b oraz max_iter mogą być zmieniane

In [28]:
n = 20
c = zeros(Float64,(n+1))
for i = 1:(n+1)
    c[i] = (Float64(i-1))^(Float64(0.5))
end

In [29]:
c_f = zeros(Float64,(n+1))
for i = 1:(n+1)
    c_f[i] = fluct((Float64(i-1))^(Float64(0.5)))
end

In [30]:
a0 = Float64(0)
a = a0
b = Float64(10)
max_iter = 50
e = (b-a0)/max_iter
res2 = []
for i = 0:max_iter
    y0 = clenshaw(n,c,a)
    y = clenshaw(n,c_f,a)
    s = rel_err(y,y0)
    push!(res2,s)
    a += e
end

In [31]:
x2 = a0:e:b
plot(x2,res2,ylim = (0,5*10^-15),seriestype=:scatter,title="Błąd względny dla dokładnych i zaburzonych ck, n=20")
xlabel!("x")
ylabel!("Błąd względny")

Funkcja pomocnicze t(n,k) oraz u(n,k)

In [14]:
function t(n,k)
    return Float64(cos(((2*k+1)/(2*n))*pi))
end

t (generic function with 1 method)

In [15]:
function u(n,k)
    return Float64(cos(((k)/(n+1))*pi))
end

u (generic function with 1 method)

Wielomian interpolacyjny In

Funkcja obliczająca macierz n x n wartości Ti(t(n+1,j))

In [16]:
function tkMatrix(n)
    matrix = ones(Float64,n+1,n+1)
    for i = 1:(n+1)
        matrix[2,i] = t(n+1,i-1)
    end
    for i = 3:(n+1)
        for j = 1:(n+1)
            matrix[i,j] = 2 * matrix[2,j] * matrix[i-1,j] - matrix[i-2,j]
        end
    end
    return matrix
end

tkMatrix (generic function with 1 method)

Funkcja, którą będziemy przybliżać (można użyć dowolnej, określonej na danym przedziale)

In [17]:
function f(x)
    return Float64(x^22 + e^x + x^7)
end

f (generic function with 1 method)

In [18]:
n = 20
c2 = zeros(Float64,(n+1))
tks = tkMatrix(n)
for i = 1:(n+1)
    s = 0
    for j = 0:n
        s += f(t((n+1),j))*tks[i,(j+1)]
    end
    c2[i] = s
end
c2_f = zeros(Float64,(n+1))
for i = 1:(n+1)
    c2_f[i] = fluct(c2[i])
end

In [19]:
a0 = Float64(-20)
a = a0
b = Float64(20)
max_iter = 100
e = (b-a0)/max_iter
res3 = []
for i = 0:max_iter
    y0 = (2/(n+1))*clenshaw(n,c2,a)
    y = (2/(n+1))*clenshaw(n,c2_f,a)
    s = rel_err(y,y0)
    push!(res3,s)
    a += e
end

In [20]:
x3 = a0:e:b
plot(x3,res3,seriestype=:scatter,title="Błąd względny dla dokładnych i zaburzonych ck, n=20")
xlabel!("x")
ylabel!("Błąd względny")

Wielomian interpolacyjny Jn

In [38]:
n = 20
c3 = zeros(Float64,(n+1))
cp = zeros(Float64,(n+1))
for i = 1:(n+1)
    cp[i] = f(u(n-1,(i-1)))
end
for i = 1:(n+1)
    c3[i] = clenshaw(n,cp,u(n-1,(i-1))) - f(-1) * czebyszew2(n,-1)
end
c3_f = zeros(Float64,(n+1))
for i = 1:(n+1)
    c3_f[i] = fluct(c3[i])
end

In [39]:
a0 = Float64(-20)
a = a0
b = Float64(20)
max_iter = 100
e = (b-a0)/max_iter
res4 = []
for i = 0:max_iter
    y0 = (2/n)*clenshaw(n,c3,a)
    y = (2/n)*clenshaw(n,c3_f,a)
    s = rel_err(y,y0)
    push!(res4,s)
    a += e
end

In [40]:
x4 = a0:e:b
plot(x4,res4,seriestype=:scatter,title="Błąd względny dla dokładnych i zaburzonych ck, n=20")
xlabel!("x")
ylabel!("Błąd względny")